In [42]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import librosa
import glob
from data_loading_preparation import DataPrep
import matplotlib
import matplotlib as mpl
from operator import itemgetter
import torch
import torch.nn as nn

In [43]:
import os
import json

path="experiments_rnn"
results = []
for el in os.listdir(path):
    if el.endswith("json"):
        with open(path+"/"+el) as f:
            report = json.load(f)
            report["filename"] = el
            if 'LSTM' in el:
                report['model'] = "LSTM"
            else:
                report['model'] = "RNN"
            if 'one' in report['name']:
                report['group'] = 'one'
            elif 'equal' in report['name']:
                report['group'] = 'equal'
            elif 'differentiator' in report['name']:
                report['group'] = 'differentiator'
            elif 'detector' in report['name']:
                report['group'] = 'detector'
            else:
                pass
            report['run_type'] = str.split(report['name'], "_")[-1]
            results.append(report)


import pandas as pd
res_df = pd.json_normalize(results, sep='.')
res_df["epochs"] = res_df["trajectory"].apply(len)

res_df["model_filename"] = res_df["filename"].apply(lambda x: x.replace("report.json", "model.pt"))

In [44]:
top10_models = res_df[res_df["group"] == "one"].sort_values("best_accuracy_validation", ascending=False).head(10)

In [45]:
top10_models

,name,run,best_accuracy_validation,time_generated,trajectory,filename,model,group,run_type,train_params.hidden_size,train_params.input_size,train_params.num_layers,train_params.classifier_size,epochs,model_filename
66,trainer_oneLSTM_params_tiny,1,0.590323,2023-04-15T13:23:33.091659,"[{'epoch': 1, 'validation': 0.2114937671667018...",trainer_oneLSTM_params_tiny_run_1_report.json,LSTM,one,tiny,40,20,1,NaN,53,trainer_oneLSTM_params_tiny_run_1_model.pt
136,trainer_oneLSTM_params_tiny,4,0.585030,2023-04-15T15:16:23.324978,"[{'epoch': 1, 'validation': 0.1999575911789652...",trainer_oneLSTM_params_tiny_run_4_report.json,LSTM,one,tiny,40,20,1,NaN,46,trainer_oneLSTM_params_tiny_run_4_model.pt
316,trainer_oneLSTM_params_tiny,3,0.578568,2023-04-15T14:28:39.975593,"[{'epoch': 1, 'validation': 0.2100804743752647...",trainer_oneLSTM_params_tiny_run_3_report.json,LSTM,one,tiny,40,20,1,NaN,66,trainer_oneLSTM_params_tiny_run_3_model.pt
130,trainer_oneLSTM_params_tiny,7,0.575112,2023-04-15T21:22:19.825791,"[{'epoch': 1, 'validation': 0.2028553164287236...",trainer_oneLSTM_params_tiny_run_7_report.json,LSTM,one,tiny,40,20,1,NaN,61,trainer_oneLSTM_params_tiny_run_7_model.pt
74,trainer_oneLSTM_params_tinydouble,10,0.560613,2023-04-16T10:57:07.591818,"[{'epoch': 1, 'validation': 0.1680136112292641...",trainer_oneLSTM_params_tinydouble_run_10_repor...,LSTM,one,tinydouble,40,20,2,NaN,40,trainer_oneLSTM_params_tinydouble_run_10_model.pt
132,trainer_oneLSTM_params_tinydouble,6,0.550266,2023-04-15T20:45:16.602897,"[{'epoch': 1, 'validation': 0.1742826780021254...",trainer_oneLSTM_params_tinydouble_run_6_report...,LSTM,one,tinydouble,40,20,2,NaN,41,trainer_oneLSTM_params_tinydouble_run_6_model.pt
330,trainer_oneLSTM_params_tiny,8,0.550096,2023-04-15T22:04:14.611034,"[{'epoch': 1, 'validation': 0.1995300149540696...",trainer_oneLSTM_params_tiny_run_8_report.json,LSTM,one,tiny,40,20,1,NaN,55,trainer_oneLSTM_params_tiny_run_8_model.pt
156,trainer_oneLSTM_params_tiny,9,0.548791,2023-04-15T22:39:19.518939,"[{'epoch': 1, 'validation': 0.2079015544041450...",trainer_oneLSTM_params_tiny_run_9_report.json,LSTM,one,tiny,40,20,1,NaN,35,trainer_oneLSTM_params_tiny_run_9_model.pt
329,trainer_oneLSTM_params_wide,6,0.547691,2023-04-15T20:52:39.448012,"[{'epoch': 1, 'validation': 0.2097539922313336...",trainer_oneLSTM_params_wide_run_6_report.json,LSTM,one,wide,80,20,1,NaN,21,trainer_oneLSTM_params_wide_run_6_model.pt
90,trainer_oneLSTM_params_tinydouble,5,0.546638,2023-04-15T20:07:54.904748,"[{'epoch': 1, 'validation': 0.1852721451440768...",trainer_oneLSTM_params_tinydouble_run_5_report...,LSTM,one,tinydouble,40,20,2,NaN,62,trainer_oneLSTM_params_tinydouble_run_5_model.pt


In [46]:
%cd
from DeepLearning.Project2.data_loading_preparation import *

/home/kacper


In [47]:
class LSTMSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        classifier_size = 512,
        add_dropout=True
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [55]:
train, test, val = load_audio_dataloaders_validation(bs=1000, limit_11=1.0)

In [56]:
def eval_on_data_one(model, dataloader, device="cuda"):
    results = []
    model.eval()
    with torch.no_grad():
        model.to(device)
        for inputs in dataloader:
            if type(inputs) == list:
                inputs, _ = inputs
            inputs = inputs.to(device)
            pred = torch.argmax(model(inputs), dim=1)
            for p in pred.cpu().numpy():
                results.append(p)
            
    model.train()
    return results

In [53]:
def load_and_eval_model(model_name, dataloader, device="cuda"):
    model = torch.load("experiments_rnn/" + model_name)
    model = model.to(device)
    model = model.eval()
    return eval_on_data_one(model, dataloader)

In [92]:
def get_test_audio_dataset(path=''):
    path = ''
    if len(path) > 0 and path[-1] != '/':
        path += '/'
    # GET FULL DATASET
    files = glob.glob(path + '.data-audioset/test/audio/*/*.wav')
    full_data = pd.DataFrame(pd.Series(files).apply(lambda x: str.replace(x, '\\', '/')))
    full_data = full_data.rename({0: 'file'}, axis=1)
    full_data["label"] = 11
    dataset = AudioDataset(full_data, 'cached_test_kaggle')
    if not os.path.exists('.data-audioset/cached_test_kaggle'):
        os.makedirs('.data-audioset/cached_test_kaggle')
    return dataset

In [93]:
def load_test_dataloader(path='', bs=16):
    dataset = get_test_audio_dataset(path=path)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs)
    return dataloader

In [102]:
test = load_test_dataloader(bs=1000)
df = pd.DataFrame()
df["fname"] = test.dataset.data["file"].apply(lambda x: x.split("/")[4])

In [104]:
for model_filename in top10_models["model_filename"]:
    df[model_filename] = load_and_eval_model(model_filename, test)

In [105]:
df

,fname,trainer_oneLSTM_params_tiny_run_1_model.pt,trainer_oneLSTM_params_tiny_run_4_model.pt,trainer_oneLSTM_params_tiny_run_3_model.pt,trainer_oneLSTM_params_tiny_run_7_model.pt,trainer_oneLSTM_params_tinydouble_run_10_model.pt,trainer_oneLSTM_params_tinydouble_run_6_model.pt,trainer_oneLSTM_params_tiny_run_8_model.pt,trainer_oneLSTM_params_tiny_run_9_model.pt,trainer_oneLSTM_params_wide_run_6_model.pt,trainer_oneLSTM_params_tinydouble_run_5_model.pt
0,clip_ad93b1782.wav,4,5,5,5,4,5,4,5,11,5
1,clip_cb4f0bd63.wav,11,5,0,11,11,5,11,11,11,11
2,clip_802099ea8.wav,2,2,2,2,2,2,2,7,2,2
3,clip_45d57c6c7.wav,6,6,6,6,6,6,6,6,6,6
4,clip_37c697d86.wav,4,4,4,4,7,4,7,7,4,4
...,...,...,...,...,...,...,...,...,...,...,...
158533,clip_44d899b10.wav,8,11,8,11,11,11,8,11,8,11
158534,clip_31c61c448.wav,11,8,8,11,11,11,9,11,9,8
158535,clip_77c442948.wav,11,11,11,11,11,11,11,11,11,11
158536,clip_65cd65b16.wav,10,10,10,10,10,4,4,10,10,10


In [107]:
from collections import Counter
df["label"] = [
    Counter(df.iloc[i,1:]).most_common()[0][0]
    for i in range(158538)
]

In [108]:
df

,fname,trainer_oneLSTM_params_tiny_run_1_model.pt,trainer_oneLSTM_params_tiny_run_4_model.pt,trainer_oneLSTM_params_tiny_run_3_model.pt,trainer_oneLSTM_params_tiny_run_7_model.pt,trainer_oneLSTM_params_tinydouble_run_10_model.pt,trainer_oneLSTM_params_tinydouble_run_6_model.pt,trainer_oneLSTM_params_tiny_run_8_model.pt,trainer_oneLSTM_params_tiny_run_9_model.pt,trainer_oneLSTM_params_wide_run_6_model.pt,trainer_oneLSTM_params_tinydouble_run_5_model.pt,label
0,clip_ad93b1782.wav,4,5,5,5,4,5,4,5,11,5,5
1,clip_cb4f0bd63.wav,11,5,0,11,11,5,11,11,11,11,11
2,clip_802099ea8.wav,2,2,2,2,2,2,2,7,2,2,2
3,clip_45d57c6c7.wav,6,6,6,6,6,6,6,6,6,6,6
4,clip_37c697d86.wav,4,4,4,4,7,4,7,7,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
158533,clip_44d899b10.wav,8,11,8,11,11,11,8,11,8,11,11
158534,clip_31c61c448.wav,11,8,8,11,11,11,9,11,9,8,11
158535,clip_77c442948.wav,11,11,11,11,11,11,11,11,11,11,11
158536,clip_65cd65b16.wav,10,10,10,10,10,4,4,10,10,10,10


In [110]:
rev_name_dictionary = {
    v: k
    for k, v in name_dict.items()
}
rev_name_dictionary[11] = "unknown"
rev_name_dictionary

{0: 'yes',
 1: 'no',
 2: 'up',
 3: 'down',
 4: 'left',
 5: 'right',
 6: 'on',
 7: 'off',
 8: 'stop',
 9: 'go',
 10: 'silence',
 11: 'unknown'}

In [111]:
df["label"] = [rev_name_dictionary[result] for result in df["label"]]

In [113]:
df[["fname", "label"]].to_csv("rnn1.csv", index=False)